# This notebook will be mainly used for the capstone project

## Introduction

Because of the population increase, there are more and more car drivers appears making the road traffic become more complicated. This leads to increase the car incident rate. So, In this project, I would like to study the relationship among level of severity of car incidents  and factors such as weather,  number of vehicles involved in the collision, the drivers are drunk or not, and so on. after this project, we can know about when should we pay more attention when driving to decrease the car incident.

And my aimed target audiances are all different types of drivers around the world

## Data Understaning

The dataset is about the collision records during 2004 to present.
it has total 38 columns in the dataset but some of them may not useful for the prediction. Also, there is labeled data called SEVERITYCODE. Therefore, there are total 37 attributes and 1 labeled data. So we do not explain all of them in details. The under are the description of the data all we need in this project.
In addiction, some attributes have missing data and categorical types of data that we need to dealing with them in the data-preprocessing stage.


The labeled data:
SEVERITYCODE : corresponds to the severity of thecollision

• 3—fatality

• 2b—serious injury

• 2—injury

• 1—prop damage

• 0—unknown


Attributes:
OBJECTID: ObjectID ESRI unique identifier 

ADDRTYPE: Collision address type

SEVERITYDESC: A detailed description of the severity of the collision

COLLISIONTYPE: Collision type

INATTENTIONIND: Whether or not collision was due to inattention.(Y/N)

UNDERINFL:Whether or not a driver involved was under the influence of drugs or alcohol.

SPEEDING: Whether or not speeding was a factor in the collision.(Y/N)

WEATHER: A description of the weather conditions during the time of the collision.

ROADCOND: The condition of the road during the collision. 

and so on...

For futher attribute information, please go to the under link
https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf

In [17]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')


Hello Capstone Project Course!


In [16]:
df = pd.read_csv(r'C:\Users\user\Downloads\Data-Collisions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEVERITYCODE    194673 non-null  int64  
 1   X               189339 non-null  float64
 2   Y               189339 non-null  float64
 3   OBJECTID        194673 non-null  int64  
 4   INCKEY          194673 non-null  int64  
 5   COLDETKEY       194673 non-null  int64  
 6   REPORTNO        194673 non-null  object 
 7   STATUS          194673 non-null  object 
 8   ADDRTYPE        192747 non-null  object 
 9   INTKEY          65070 non-null   float64
 10  LOCATION        191996 non-null  object 
 11  EXCEPTRSNCODE   84811 non-null   object 
 12  EXCEPTRSNDESC   5638 non-null    object 
 13  SEVERITYCODE.1  194673 non-null  int64  
 14  SEVERITYDESC    194673 non-null  object 
 15  COLLISIONTYPE   189769 non-null  object 
 16  PERSONCOUNT     194673 non-null  int64  
 17  PEDCOUNT  

In [18]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [14]:
df['ROADCOND'].isnull().sum()

5012